In [80]:
import os
import sys
import requests
import re
import time
import json
import pandas as pd
import csv
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

In [81]:
extracts = re.compile('[^ 가-힣|a-z|A-Z|0-9|\[|\]|(|)|-|~|?|!|.|,|:|;|%]+')
cost_extracts = re.compile('[^ 가-힣|a-z|A-Z|\[|\]|(|)|-|~|?|!|.|,|:|;|%]+')

In [82]:
# 서울
data_url = open('./data/goodchoice_seoul.csv', 'w', encoding='utf8') # url 저장 관련 csv파일
data = open('./data/info_seoul.csv', 'w', encoding='utf8') # 정보 저장관련 csv 파일

# 부산
# data_url = open('./data/goodchoice_busan.csv', 'w', encoding='utf8') # url 저장 관련 csv파일
# data = open('./data/info_busan.csv', 'w', encoding='utf8') # 정보 저장관련 csv 파일

In [83]:
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window() #전체 화면
Url = 'https://www.goodchoice.kr/product/search/1/7052' #서울 역삼, 강남, 논현
# Url = 'https://www.goodchoice.kr/product/search/1/7044' #부산 광안리, 수영, 민락
driver.get(Url)

In [84]:
urls = driver.find_element(By.ID, "poduct_list_area")
urlList = urls.find_elements(By.TAG_NAME, 'li')

for li in urlList:
    aTag = li.find_element(By.TAG_NAME, 'a')
    href = aTag.get_attribute('href')
    data_url.write(href)
    data_url.write('\n')

data_url.close()
    

In [85]:
data.write('"basic_info"')
data.write(', ')
data.write('"cost_info"')
data.write(', ')
data.write('"stars"')
data.write('\n')

1

In [86]:
# 서울
f = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/data/goodchoice_seoul.csv', 'r', encoding='utf8')

# 부산
# f = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/data/goodchoice_busan.csv', 'r', encoding='utf8')
rdr = csv.reader(f)

In [87]:
# 기본 정보 및 요금 정보
for line in rdr:
    addr = line[0]
    rs = requests.get(addr)
    rs.encoding = None
    html = rs.text
    soup = bs(html, 'html.parser')

    try:
        tarbasic_info = soup.select('#content > article.detail_info > section.default_info')
        basic_info = ''
        for text in tarbasic_info:
            basic_info += text.getText()

        basic_info = extracts.sub('', basic_info)
        basic_info = re.sub(' +', ' ', basic_info)
        basic_info = '"'+basic_info+'"'


        tarcost_info = soup.select('#content > article.detail_info.on > section.table_wrap > div:nth-child(2) > table:nth-child(2)')
        rcost_info = soup.find("table", {"class" : "table_type_02"})
        cost_info = rcost_info.find("span")
        cost_info = cost_info.text


        tarstar_info = soup.select('#content > div.top > div.right > div.info')
        rstar_info = soup.find("div", {"class" : "score_cnt"})
        star_info = rstar_info.find("span").text[0:4]

        data.write(basic_info)
        data.write(', ')
        data.write(cost_info)
        data.write(', ')
        data.write(star_info)
        data.write('\n')
    except:
        data.write("Error")
        data.write('\n')
        continue

f.close()
data.close()